In [32]:
# Import libraries
import sqlalchemy
import pandas as pd
from  psycopg2 import connect
import os
from config import *
from sqlalchemy import Column, ForeignKey,Table, Integer, String, Float, Boolean, MetaData, create_engine

In [21]:
# Input file paths
cleaned_tweet_file=os.path.join("","..","Resources","outputData","tweetCleandata.csv")
predict_tweet_file=os.path.join("","..","Resources","outputData","tweetPredData.csv")

In [22]:
# Load tweet csv file
tweet_clean_df= pd.read_csv(cleaned_tweet_file)
tweet_clean_df.head()

,TweetID,Date,Matched Keywords,User,Source,Followers,Friends,Favorite,Tweet,predTweet,CleanedTweet
0,1290598653770575872,08/04/2020,Trump,genadamedia,GenadaMedia,1685.0,1642.0,0.0,Trump: TikTok Must Sell Its American Operation...,trump tiktok must sell its american operations...,"['trump', 'tiktok', 'american', 'over']"
1,1290598652847816706,08/04/2020,Trump,Carolin64234118,Twitter for Android,0.0,41.0,0.0,@richardmarx I need whatever the reporter is t...,need whatever the reporter taking keep his coo...,"['whaten', 'report', 'take', 'exercise', 'pati..."
2,1290598651966951424,08/04/2020,Trump,soulb4time,Twitter Web App,42.0,117.0,0.0,@GOPChairwoman President Trump & the RN keep s...,president trump and the rec keep sending mail ...,"['preside', 'trump', 'send', 'decease', 'husba..."
3,1290598649626599424,08/04/2020,Trump,Jan714,Twitter Web App,39.0,120.0,0.0,"@realDonaldTrump \n""Donald Trump dumped $400 m...",donald trump dumped million into his clubs in ...,"['donald', 'trump', 'dump', 'million', 'club',..."
4,1290598646740848640,08/04/2020,Trump,jocfanaccount,Twitter for iPhone,659.0,730.0,0.0,This is how every single journalist should be ...,this how every single journalist should be tal...,"['every', 'single', 'journalist', 'talk', 'eve..."


In [23]:
# Load tweet csv file
tweet_predict_df= pd.read_csv(predict_tweet_file)
tweet_predict_df.head()

,TweetID,Date,Matched Keywords,User,Source,Followers,Friends,Favorite,Tweet,predTweet,CleanedTweet,Prediction
0,1290598653770575872,08/04/2020,Trump,genadamedia,GenadaMedia,1685.0,1642.0,0.0,Trump: TikTok Must Sell Its American Operation...,trump tiktok must sell its american operations...,"['trump', 'tiktok', 'american', 'over']",Negative
1,1290598652847816706,08/04/2020,Trump,Carolin64234118,Twitter for Android,0.0,41.0,0.0,@richardmarx I need whatever the reporter is t...,need whatever the reporter taking keep his coo...,"['whaten', 'report', 'take', 'exercise', 'pati...",Positive
2,1290598651966951424,08/04/2020,Trump,soulb4time,Twitter Web App,42.0,117.0,0.0,@GOPChairwoman President Trump & the RN keep s...,president trump and the rec keep sending mail ...,"['preside', 'trump', 'send', 'decease', 'husba...",Negative
3,1290598649626599424,08/04/2020,Trump,Jan714,Twitter Web App,39.0,120.0,0.0,"@realDonaldTrump \n""Donald Trump dumped $400 m...",donald trump dumped million into his clubs in ...,"['donald', 'trump', 'dump', 'million', 'club',...",Negative
4,1290598646740848640,08/04/2020,Trump,jocfanaccount,Twitter for iPhone,659.0,730.0,0.0,This is how every single journalist should be ...,this how every single journalist should be tal...,"['every', 'single', 'journalist', 'talk', 'eve...",Negative


In [24]:
# Create table specific data frames

In [25]:
tweet_user_df= tweet_clean_df[['TweetID','Date','Matched Keywords','User','Followers','Friends','Favorite']]

In [26]:
tweet_source_df = tweet_clean_df[['TweetID','Date','Matched Keywords','Source']]

In [27]:
tweet_df = tweet_clean_df[['TweetID','Date','Matched Keywords','Tweet']]

In [28]:
tweet_clean_df = tweet_clean_df[['TweetID','Date','Matched Keywords','CleanedTweet']]

In [29]:
tweet_predict_df = tweet_predict_df[['TweetID','Date','Matched Keywords','predTweet','Prediction']]

In [36]:
# Database connection
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbName}')

In [37]:
# Create MetaData
meta = MetaData(engine)

In [56]:
# 1. Twitter User Table
twitter_user = Table('tweetUser', meta,
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('User', String),
    Column('Followers', Integer),
    Column('Friends', Integer),
    Column('Favorite', Integer))

meta.create_all(engine)

tweet_user_df.to_sql(name='tweetUser', con=engine, if_exists='append', index=False)

In [51]:
# 2. Twitte Source Table
twitter_source = Table('twitterSource', meta, 
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('Source', String))

meta.create_all(engine)

tweet_source_df.to_sql(name='twitterSource', con=engine, if_exists='append', index=False)

In [53]:
# 3. Twitte Data Table
twitter_data = Table('tweetsData', meta, 
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('Tweet', String))

meta.create_all(engine)

tweet_df.to_sql(name='tweetsData', con=engine, if_exists='append', index=False)

In [54]:
# 4. Twitter Clean Data Table
twitter_clean_data = Table('tweetsCleanData', meta, 
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('CleanedTweet', String))

meta.create_all(engine)

tweet_clean_df.to_sql(name='tweetsCleanData', con=engine, if_exists='append', index=False)

In [55]:
# 5. Twitter Prediction Data Table
twitter_predict_data = Table('tweetsPredictData', meta, 
    Column('id', Integer, primary_key = True), 
    Column('TweetID', String), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('predTweet', String),
    Column('Prediction', String))

meta.create_all(engine)

tweet_predict_df.to_sql(name='tweetsPredictData', con=engine, if_exists='append', index=False)

In [57]:
engine.table_names()

['tweetUser',
 'twitterSource',
 'tweetsData',
 'tweetsCleanData',
 'tweetsPredictData']